In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib
import json

import IPython.display

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scipy.interpolate
import scipy.signal

import pymedphys
import pymedphys._wlutz.pylinac
import pymedphys._wlutz.iview
import pymedphys._wlutz.imginterp
import pymedphys._wlutz.findfield
import pymedphys._wlutz.findbb
import pymedphys._wlutz.reporting

In [ ]:
penumbra = 2
edge_lengths = [20, 24]
initial_rotation = 0
bb_diameter = 8

pd.set_option("display.max_rows", 101)

In [ ]:
frame_paths_list = pymedphys.zip_data_paths("wlutz_arc_session.zip", check_hashes=False)

frame = [path.stem.split('_')[1] for path in frame_paths_list]
timestamps = [path.parent.stem for path in frame_paths_list]
directions = [path.parent.parent.stem for path in frame_paths_list]
beams = [path.parent.parent.parent.stem for path in frame_paths_list]

keys = list(zip(beams, directions, timestamps, frame))

image_paths = {
    key: path for key, path in zip(keys, frame_paths_list)
}

In [ ]:
key_map = {
    key: '-'.join(key) for key in keys
}
inv_key_map = {
    item: key for key, item in key_map.items()
}

In [ ]:
# data = {}

In [ ]:
try:
    with open('session_cache.json', 'r') as a_file:
        data_string_keys = json.load(a_file)
    
    data = {
        inv_key_map[key]: item for key, item in data_string_keys.items()
    }
except FileNotFoundError:
    data = {}

In [ ]:
for key, image_path in image_paths.items():
    try:
        this_data = data[key]
        pymedphys_data = this_data['pymedphys']
    except KeyError:
        this_data = {}
        pymedphys_data = {}
        this_data['pymedphys'] = pymedphys_data
        data[key] = this_data
    
    try:
        pymedphys_data['field_centre']
        pymedphys_data['field_rotation']
        this_data['pylinac']
        pymedphys_data['bb_centre']
    except KeyError:
        print(key)
        x, y, img = pymedphys._wlutz.iview.iview_image_transform(image_path)

        field = pymedphys._wlutz.imginterp.create_interpolated_field(x, y, img)
        initial_centre = pymedphys._wlutz.findfield._initial_centre(x, y, img)
    
    try:
        pymedphys_data['field_centre']
        pymedphys_data['field_rotation']
    except KeyError:
    
        try:
            pymedphys_data['field_centre'], pymedphys_data['field_rotation'] = pymedphys._wlutz.findfield.field_centre_and_rotation_refining(
                field, edge_lengths, penumbra, initial_centre, initial_rotation=initial_rotation
            )
        except ValueError as e:
            print(e)
            continue  
        
        pymedphys_data['field_centre'] = pymedphys_data['field_centre']
        pymedphys_data['field_rotation'] = pymedphys_data['field_rotation']
    
    try:
        this_data['pylinac']
    except KeyError:
        try:
            this_data['pylinac'] = pymedphys._wlutz.pylinac.run_wlutz(
                field, edge_lengths, penumbra, pymedphys_data['field_centre'], pymedphys_data['field_rotation'])
        except Exception as e:
            print(e)
            pass
    
    
    try:
        pymedphys_data['bb_centre']
    except KeyError:
        try:
            pymedphys_data['bb_centre'] = pymedphys._wlutz.findbb.optimise_bb_centre(
                field, bb_diameter, edge_lengths, penumbra, pymedphys_data['field_centre'], pymedphys_data['field_rotation']
            )
        except ValueError as e:
            print(e)
            continue
    
        pymedphys_data['bb_centre'] = pymedphys_data['bb_centre']
    
    
#     try:
#         fig = pymedphys._wlutz.reporting.image_analysis_figure(
#             x,
#             y,
#             img,
#             this_data['bb_centre'],
#             this_data['field_centre'],
#             this_data['field_rotation'],
#             bb_diameter,
#             edge_lengths,
#             penumbra,
#         )

#         plt.title('PyMedPhys Basinhopping Method')
#     except KeyError:
#         pass
    
#     try:
#         fig = pymedphys._wlutz.reporting.image_analysis_figure(
#             x,
#             y,
#             img,
#             this_data['pylinac']['v2.2.6']['bb_centre'],
#             this_data['pylinac']['v2.2.6']['field_centre'],
#             this_data['field_rotation'],
#             bb_diameter,
#             edge_lengths,
#             penumbra,
#         )

#         plt.title('Pylinac v2.2.6 Filter and Profile Method')
#     except KeyError:
#         pass
    
#     try:
#         fig = pymedphys._wlutz.reporting.image_analysis_figure(
#             x,
#             y,
#             img,
#             this_data['pylinac']['v2.2.7']['bb_centre'],
#             this_data['pylinac']['v2.2.7']['field_centre'],
#             this_data['field_rotation'],
#             bb_diameter,
#             edge_lengths,
#             penumbra,
#         )

#         plt.title('Pylinac v2.2.7 Filter and Scikit-Image Method')
#     except KeyError:
#         pass
    
    
#     plt.show()

In [ ]:
data_for_json = {
    key_map[key]: item for key, item in data.items()
}

with open('session_cache.json', 'w') as a_file:
    json.dump(data_for_json, a_file, indent=2)
